In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import time

# 레시피명과 URL 정보를 저장할 리스트 초기화
recipe_name_list = []
url_list = []
view_list = []
img_list = []
recipe_img_list = []

# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# ChromeDriverManager 자동설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # 페이지를 순회하면서 데이터 수집
    for page_num in range(1, 10):  # 1부터 101페이지까지
        
        url = f'https://www.10000recipe.com/recipe/list.html?order=reco&page={page_num}'
        driver.get(url)
        time.sleep(1)
        
        WebDriverWait(driver, 10)  # 웹 페이지가 로드될 때까지, 최대 10초간 기다린다는 뜻

        # Selenium으로 javascript 실행된 후의 페이지 소스를 가져옴
        html = driver.page_source
        
        # Beautifulsoup 으로 파싱
        soup = BeautifulSoup(html, 'html.parser')
        
        # box_contents 찾기
        box_contents = soup.find_all('li', class_='common_sp_list_li')
        
        # 이하 데이터 처리 로직...
        print(f"Page {page_num}: Found {len(box_contents)} box_contents")
        
        for box_content in box_contents:
            # 조회수 데이터 추출
            view_data = box_content.find('span', class_='common_sp_caption_buyer').text.split()[1]
            
            # '만'을 포함하거나, '만'이 없으면서 조회수가 5,000 이상인 경우
            if '만' in view_data:
                should_add = True
                view_list.append(view_data)
            else:
                # 쉼표 제거 후 숫자형으로 변환 가능한지 확인하고, 5,000 이상인 경우 리스트에 추가
                view_count = view_data.replace(',', '')
                if view_count.isdigit() and int(view_count) >= 5000:
                    should_add = True
                    view_list.append(view_data)
                else:
                    should_add = False
                    
            # 조회수 조건을 충족하는 경우에만 img_link 정보를 리스트에 추가
            # 레시피 이미지 링크 수집을 위한 이미지 리스트 초기화
            img_list = [] 
        
            if should_add:
                img_tags = box_content.find_all('a', class_='common_sp_link')
                for img_tag in img_tags:
                    img_links = img_tag.select('img')  
                    for img_link in img_links: 
                        img_list.append(img_link['src'])
                        
            # 레시피 이미지를 포함하는 링크 정보를 recipe_img_list에 추가
            # 처음에는 문자열 'm.jpg'를 포함하는 링크 정보를 추가하였으나 결과값이 39개 나와 문자열'cache'로 수정                     
                for recipe_link in img_list:
                    if 'cache' in recipe_link:
                        recipe_img_list.append(recipe_link) 
                    
            # 조회수 조건을 충족하는 경우에만 레시피명과 URL 정보를 리스트에 추가
            if should_add:
                recipe_name = box_content.find('div', class_='common_sp_caption_tit line2').text.strip()
                url = "https://www.10000recipe.com" + box_content.find('a')['href']
                recipe_name_list.append(recipe_name)
                url_list.append(url)
                
            # 각 리스트별 개수가 같은지 확인
            print(f"레시피 이름 {len(recipe_name_list)}개 / url주소 {len(url_list)}개 / 조회수 {len(view_list)}개 / 레시피 이미지 {len(recipe_img_list)}개")
                
        # 각 리스트별 리스트 개수가 쌓이는 개수를 확인
        print(f"레시피 이름 {len(recipe_name_list)}개 / url주소 {len(url_list)}개 / 조회수 {len(view_list)}개 / 레시피 이미지 {len(recipe_img_list)}개")
    # 최종개수를 확인
    print(f" 최종개수 : 레시피 이름 {len(recipe_name_list)}개 / url주소 {len(url_list)}개 / 조회수 {len(view_list)}개 / 레시피 이미지 {len(recipe_img_list)}개")
            
finally:
    # 드라이버 종료는 모든 작업이 끝난 후에 한 번만 수행
    driver.quit()
 
# 최종개수를 확인
print(f" 최종개수 : 레시피 이름 {len(recipe_name_list)}개 / url주소 {len(url_list)}개 / 조회수 {len(view_list)}개 / 레시피 이미지 {len(recipe_img_list)}개")      

# 데이터 저장
data = {'name':recipe_name_list, 'link':url_list, 'viewNumber': view_list, 'imageURL':recipe_img_list}
df = pd.DataFrame(data)
        
df.to_csv('만개의레시피(3290개).csv', index=False, encoding='utf-8-sig')

Page 1: Found 40 box_contents
레시피 이름 1개 / url주소 1개 / 조회수 1개 / 레시피 이미지 1개
레시피 이름 1개 / url주소 1개 / 조회수 1개 / 레시피 이미지 1개
레시피 이름 1개 / url주소 1개 / 조회수 1개 / 레시피 이미지 1개
레시피 이름 2개 / url주소 2개 / 조회수 2개 / 레시피 이미지 2개
레시피 이름 3개 / url주소 3개 / 조회수 3개 / 레시피 이미지 3개
레시피 이름 3개 / url주소 3개 / 조회수 3개 / 레시피 이미지 3개
레시피 이름 4개 / url주소 4개 / 조회수 4개 / 레시피 이미지 4개
레시피 이름 5개 / url주소 5개 / 조회수 5개 / 레시피 이미지 5개
레시피 이름 5개 / url주소 5개 / 조회수 5개 / 레시피 이미지 5개
레시피 이름 6개 / url주소 6개 / 조회수 6개 / 레시피 이미지 6개
레시피 이름 7개 / url주소 7개 / 조회수 7개 / 레시피 이미지 7개
레시피 이름 8개 / url주소 8개 / 조회수 8개 / 레시피 이미지 8개
레시피 이름 8개 / url주소 8개 / 조회수 8개 / 레시피 이미지 8개
레시피 이름 8개 / url주소 8개 / 조회수 8개 / 레시피 이미지 8개
레시피 이름 8개 / url주소 8개 / 조회수 8개 / 레시피 이미지 8개
레시피 이름 9개 / url주소 9개 / 조회수 9개 / 레시피 이미지 9개
레시피 이름 9개 / url주소 9개 / 조회수 9개 / 레시피 이미지 9개
레시피 이름 10개 / url주소 10개 / 조회수 10개 / 레시피 이미지 10개
레시피 이름 11개 / url주소 11개 / 조회수 11개 / 레시피 이미지 11개
레시피 이름 11개 / url주소 11개 / 조회수 11개 / 레시피 이미지 11개
레시피 이름 11개 / url주소 11개 / 조회수 11개 / 레시피 이미지 11개
레시피 이름 11개 / url주소 11개 / 조회수 11개 / 레시피 이미지 11개
레시피 

Page 6: Found 40 box_contents
레시피 이름 151개 / url주소 151개 / 조회수 151개 / 레시피 이미지 151개
레시피 이름 152개 / url주소 152개 / 조회수 152개 / 레시피 이미지 152개
레시피 이름 153개 / url주소 153개 / 조회수 153개 / 레시피 이미지 153개
레시피 이름 154개 / url주소 154개 / 조회수 154개 / 레시피 이미지 154개
레시피 이름 155개 / url주소 155개 / 조회수 155개 / 레시피 이미지 155개
레시피 이름 156개 / url주소 156개 / 조회수 156개 / 레시피 이미지 156개
레시피 이름 157개 / url주소 157개 / 조회수 157개 / 레시피 이미지 157개
레시피 이름 158개 / url주소 158개 / 조회수 158개 / 레시피 이미지 158개
레시피 이름 159개 / url주소 159개 / 조회수 159개 / 레시피 이미지 159개
레시피 이름 160개 / url주소 160개 / 조회수 160개 / 레시피 이미지 160개
레시피 이름 161개 / url주소 161개 / 조회수 161개 / 레시피 이미지 161개
레시피 이름 162개 / url주소 162개 / 조회수 162개 / 레시피 이미지 162개
레시피 이름 163개 / url주소 163개 / 조회수 163개 / 레시피 이미지 163개
레시피 이름 164개 / url주소 164개 / 조회수 164개 / 레시피 이미지 164개
레시피 이름 165개 / url주소 165개 / 조회수 165개 / 레시피 이미지 165개
레시피 이름 166개 / url주소 166개 / 조회수 166개 / 레시피 이미지 166개
레시피 이름 167개 / url주소 167개 / 조회수 167개 / 레시피 이미지 167개
레시피 이름 168개 / url주소 168개 / 조회수 168개 / 레시피 이미지 168개
레시피 이름 169개 / url주소 169개 / 조회수 169개 / 레시피 이미지 169개
레

 최종개수 : 레시피 이름 290개 / url주소 290개 / 조회수 290개 / 레시피 이미지 290개


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import UnexpectedAlertPresentException
from bs4 import BeautifulSoup
import pandas as pd
import time

# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# ChromeDriverManager 자동설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # 모든 레시피 재료 정보를 저장할  리스트
    all_recipes_ingredients = []
    all_recipes_ingredients_num = []
    review_list = []
    
    # 제거할 인덱스를 저장할 리스트
    indexes_to_remove = []
    
    recipe_number = 0
    # url_list를 돌면서 페이지마다 재료정보 가져오기
    for index, url in enumerate(url_list):
        try:
            driver.get(url)
            WebDriverWait(driver, 10)  # 웹 페이지가 로드될 때까지, 최대 10초간 기다린다는 뜻
            time.sleep(1)

            # Selenium으로 javascript 실행된 후의 페이지 소스를 가져옴
            html = driver.page_source

            # Beautifulsoup 으로 파싱
            soup = BeautifulSoup(html, 'html.parser')

            # 현재 레시피의 모든 재료를 저장할 리스트
            current_recipe_ingredients = []

            # 재료 정보에 해당하는 태그정보를 가져오기
            div_tags = soup.select_one('div.ready_ingre3') # class가 'ready_ingre3'인 div 태그 선택
            ul_tag = div_tags.select_one('ul')
            div_tags = ul_tag.find_all('div', class_='ingre_list_name')

            sauce_list = ['진간장', '통깨', '굴소스', '소금', '올리고당', '설탕', '맛간장', '맛소금' 
                          '참기름', '식용유', '후추가루', '간장', '깨소금', '식초', '후춧가루', '매실액','참치액젓','간장'
                          '멸치액젓', '들깨가루', '볶음참깨', '미림','들기름', '깨', '후추', '참치액', '흑설탕', '황설탕',''
                          '국간장', '육수', '물', '멸치육수', '올리브오일', '전분가루', '마요네즈', '매실청', '양조간장', '현미유',
                          '감자전분', '고추가루', '기름', '파슬리가루', '생수', '고춧가루', '케챱', '캐찹', '캐첩', '케첩', '캐챱',
                          '부침가루', '다진 홍고추', '다진 부추', '다진 양파', '참깨','굴소스', '올리고당', '고추장', '간장', '물', 
                          '다진마늘', '식초', '설탕', '깨', '들기름', '후춧가루',
                          '매실액', '참기름','통깨','진간장','고춧가루','후추' , '마요네즈', '머스타드소스',
                          '소금', '파슬리가루', '파슬리', '오일', '굴소스', '식용유', '밥', '맛소금',
                          '파마산치즈가루', '다이제스티브', '녹인버터', '맛술', '레몬즙', '발사믹', '바질', '된장',
                          '슬라이스아몬드' , '참치액젓', '국간장' , '다진 쪽파', '자른 다시마', '케찹',
                          '생강가루', '굵은소금', '갈은 마늘', '천일염','다진파', '머스터드',
                          '간 마늘', '검은깨', '꿀', '대파 흰부분', '파마산 치즈','깨소금', '다진청양고추', '백설탕', '통후추',
                          '물엿', '케첩', '케챱', '대파(줄기)', '다진생강 (즙)', '올리고 당', '레드페퍼', '땡초'
                          '쪽파 썰어준 것', '생크림', '미림','건바질','시나몬', '시나몬 가루', '시나몬가루',
                          '스테이크 소스', '스테이크소스', '견과류', '꽃소금', '연두', '생강술(미림맛술)', '청주', '케찹',
                          '스위트 칠리소스', '스위트칠리소스','올리브유', '카놀라유', '필링용 딸기잼', '황설탕', '흑설탕',
                          '다진홍청양고추', '다진양파', '청양고추 매콤하게 먹고싶으면', '스톡 이나 굴소스',  '다진마늘 보다적게', 
                          '참기름 보다적게', '냉장고속채소', '일반 후라이팬', '허브소금', '다진 마늘', '다진 생강', 
                          '허니머스타드', '장식용 설탕', '굵은 소금', '카놀라유 올리브유 빼고', '햇반 1개', '햇반',
                          '소주 또는 맛술', '햇반 1개', '대파 손가락 길이', '견과류 아몬드or땅콩 생략 가능',
                          '맛간장 OR진간장', '각종자투리 야채', '설탕 적은', '스리라차소스', '스리라차', '스리라차 소스',
                          '쥐똥고추 or청양고추'
                          
                         ]

            for ingre_tag in div_tags:
                a_tag = ingre_tag.find('a')
                if a_tag:  # 만약 <a> 태그가 존재하면
                    ingredient = ' '.join(a_tag.text.strip().split())  # 문자열 내의 모든 공백을 제거하고, 불필요한 공백을 최소화
                    if ingredient not in sauce_list:  # 재료가 소스 리스트에 포함되어 있지 않다면
                        current_recipe_ingredients.append(ingredient)
                    else:
                        continue  # 재료가 소스 리스트에 포함되어 있다면, 이 재료는 무시하고 넘어감. 
            recipe_number += 1 

            all_recipes_ingredients_num.append(len(current_recipe_ingredients))
            # 양념을 제외한 재료 개수, 재료정보를 확인하기 위한 용도
            print(f"재료의 개수:{len(current_recipe_ingredients)} 재료 리스트:{current_recipe_ingredients} {recipe_number}번째 레시피")
            all_recipes_ingredients.append(current_recipe_ingredients)

            # 현재 레시피의 리뷰 수 가져오기
            review_spans = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.reply_tit span'))
            )

            review_count = 0
            for span in review_spans:
                r_text = span.text.strip()  # 텍스트 앞뒤 공백 제거
                if r_text.isdigit():  # 텍스트가 숫자인지 확인
                    review_count += int(r_text)  # 숫자라면 정수로 변환하여 누적
            
            review_list.append(review_count)
        
        except TimeoutException:
            print("제한 시간 초과")
            # 제한 시간이 초과되면 해당 URL을 url_list에서 제거
            indexes_to_remove.append(index)
        except NoSuchElementException:
            print("요소를 찾을 수 없음")
            indexes_to_remove.append(index)
        except UnexpectedAlertPresentException:
            print("알 수 없는 경고가 발생")
            # 알 수 없는 경고가 발생할 경우 해당 URL을 url_list에서 제거
            indexes_to_remove.append(index)
            continue

except Exception as e:
    print("예상치 못한 오류 발생:", e)
    

finally:
    # 드라이버 종료는 모든 작업이 끝난 후에 한 번만 수행
    driver.quit()
    
# url_list에서 해당하는 인덱스를 제거
for index in sorted(indexes_to_remove, reverse=True):
    del url_list[index]
    
# img_list에서 해당하는 인덱스를 제거
for index in sorted(indexes_to_remove, reverse=True):
    del recipe_mg_list[index]

# views_list에서 해당하는 인덱스를 제거
for index in sorted(indexes_to_remove, reverse=True):
    del view_list[index]

# recipe_name_list에서 해당하는 인덱스를 제거
for index in sorted(indexes_to_remove, reverse=True):
    del recipe_name_list[index]
    
# 데이터 저장
data = {'ingredients': all_recipes_ingredients, 'ingreNumber': all_recipes_ingredients_num, 'reviewNumber': review_list} 
df1 = pd.DataFrame(data)

# df1(재료정보 데이터프레임)을 df(레시피명 및 주소정보 데이터프레임)의 왼쪽에 이어 붙임.
result = pd.concat([df, df1], axis=1)
        
# result.to_csv('C:\\Users\\gudrb\\OneDrive\\Desktop\\캡디\\만개의레시피(최종결과)_f.csv', index=False, encoding='utf-8-sig')
    

재료의 개수:2 재료 리스트:['감자', '스팸'] 1번째 레시피
재료의 개수:2 재료 리스트:['느타리버섯', '애호박'] 2번째 레시피
재료의 개수:1 재료 리스트:['콩나물'] 3번째 레시피
재료의 개수:4 재료 리스트:['감자', '베이컨', '대파', '양파'] 4번째 레시피
재료의 개수:2 재료 리스트:['미니새송이버섯', '버터'] 5번째 레시피
재료의 개수:1 재료 리스트:['오이고추'] 6번째 레시피
재료의 개수:2 재료 리스트:['새송이버섯', '대파'] 7번째 레시피
재료의 개수:2 재료 리스트:['청경채', '쪽파'] 8번째 레시피
재료의 개수:3 재료 리스트:['도토리묵', '조미김', '다진대파'] 9번째 레시피
재료의 개수:2 재료 리스트:['양파', '비엔나소세지'] 10번째 레시피
재료의 개수:3 재료 리스트:['콩나물', '크래미', '대파'] 11번째 레시피
재료의 개수:1 재료 리스트:['알배추'] 12번째 레시피
재료의 개수:1 재료 리스트:['가시오이'] 13번째 레시피
재료의 개수:2 재료 리스트:['고구마', '모짜렐라치즈'] 14번째 레시피
재료의 개수:3 재료 리스트:['참치', '양파', '청양고추'] 15번째 레시피
재료의 개수:2 재료 리스트:['잔파', '양파'] 16번째 레시피
재료의 개수:3 재료 리스트:['두부', '팽이버섯', '대파'] 17번째 레시피
재료의 개수:2 재료 리스트:['오징어채', '소주 또는 맛술'] 18번째 레시피
재료의 개수:3 재료 리스트:['계란', '양배추', '모짜렐라치즈'] 19번째 레시피
재료의 개수:3 재료 리스트:['새송이버섯', '버터', '어린잎채소'] 20번째 레시피
재료의 개수:2 재료 리스트:['순두부', '마늘'] 21번째 레시피
재료의 개수:2 재료 리스트:['베이컨', '양파'] 22번째 레시피
재료의 개수:2 재료 리스트:['햇반 1개', '참치'] 23번째 레시피
재료의 개수:4 재료 리스트:['알배추', '두부', '참치', '계란'] 24번째 

재료의 개수:4 재료 리스트:['참치', '감자 작은거', '양파', '대파'] 173번째 레시피
재료의 개수:3 재료 리스트:['후랑크 소시지', '피자치즈', '양파'] 174번째 레시피
재료의 개수:2 재료 리스트:['식빵', '캔옥수수'] 175번째 레시피
재료의 개수:4 재료 리스트:['미니새송이버섯', '통마늘', '양파', '청양고추'] 176번째 레시피
재료의 개수:4 재료 리스트:['닭다리살', '고구마', '브로콜리', '미니파프리카'] 177번째 레시피
재료의 개수:2 재료 리스트:['양파', '영양부추'] 178번째 레시피
재료의 개수:2 재료 리스트:['감자', '양파'] 179번째 레시피
재료의 개수:2 재료 리스트:['양파', '대파'] 180번째 레시피
재료의 개수:2 재료 리스트:['두부', '다진소고기'] 181번째 레시피
재료의 개수:3 재료 리스트:['전분', '계란', '크래미맛살'] 182번째 레시피
재료의 개수:2 재료 리스트:['표고버섯', '달걀'] 183번째 레시피
재료의 개수:1 재료 리스트:['냉동만두'] 184번째 레시피
재료의 개수:2 재료 리스트:['알새우', '양파'] 185번째 레시피
재료의 개수:2 재료 리스트:['두툼한 돼지고기', '튀김가루'] 186번째 레시피
재료의 개수:2 재료 리스트:['돼지고기앞다리살', '마늘'] 187번째 레시피
재료의 개수:3 재료 리스트:['닭다리살 정육', '바질가루', '버터'] 188번째 레시피
재료의 개수:2 재료 리스트:['오징어', '꽈리고추'] 189번째 레시피
재료의 개수:3 재료 리스트:['통조림햄 참치.돼지고기.등도 가능', '부추', '청양고추'] 190번째 레시피
재료의 개수:2 재료 리스트:['브로콜리', '마늘'] 191번째 레시피
재료의 개수:5 재료 리스트:['밀가루박력분', '무염버터', '우유', '베이킹파우더', '딸기잼'] 192번째 레시피
재료의 개수:7 재료 리스트:['다시마멸치육수', '만두', '달걀', '느타리버섯', '

In [3]:
# c_df = pd.read_csv('만개의레시피(최종결과)_f.csv')

In [10]:
c_df = result

In [11]:
# 'viewNumber' 컬럼의 값들을 순회하며 처리합니다.
for idx, view_num in c_df['viewNumber'].items():
    # '조회수'를 제거합니다.
    view_num_str = str(view_num)
    view_num_cleaned = view_num_str.replace('조회수', '')
    # 쉼표를 제거합니다.
    view_num_cleaned = view_num_cleaned.replace(',', '')
    # '만'이 포함된 경우에만 처리합니다.
    if '만' in view_num_cleaned:
        # '만'을 제외한 숫자 값만 추출합니다.
        num_only = float(view_num_cleaned.replace('만', '')) * 10000
    else:
        # '만'이 없는 경우에는 그냥 숫자로 변환합니다.
        num_only = float(view_num_cleaned)
    # 처리한 값을 'viewNumber' 컬럼에 업데이트합니다.
    c_df.at[idx, 'viewNumber'] = num_only

In [13]:
# 'view_Number' 열의 데이터 타입을 int로 변경합니다.
c_df['viewNumber'] = c_df['viewNumber'].astype(int)

# 'view_Number' 열에서 5000 이하의 값을 가진 행을 필터링합니다.
less_than_or_equal_to_5000 = c_df[c_df['viewNumber'] <= 5000]

# 5000 이하의 값을 가진 행이 있는지 확인합니다.
if not less_than_or_equal_to_5000.empty:
    print("5000 이하의 값이 존재합니다.")
    print(less_than_or_equal_to_5000)
else:
    print("5000 이하의 값이 존재하지 않습니다.")

5000 이하의 값이 존재하지 않습니다.


In [14]:
c_df.to_csv('C:\\Users\\gudrb\\OneDrive\\Desktop\\캡디\\만개의레시피(최종결과)_c.csv', index=False, encoding='utf-8-sig')